# Machine Learning Practice
## Module 9: Logistic Regression

Andrew H. Fagg (andrewhfagg@gmail.com)


In [2]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error
from sklearn.metrics import confusion_matrix
from sklearn.metrics import log_loss
from sklearn.metrics import roc_curve, auc
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.linear_model import SGDClassifier, LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Default figure parameters
plt.rcParams['figure.figsize'] = (8,5)
plt.rcParams['font.size'] = 12
plt.rcParams['legend.fontsize'] = 12
plt.rcParams['xtick.labelsize'] = 12
plt.rcParams['ytick.labelsize'] = 12
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['axes.titlesize'] = 18
plt.rcParams['axes.labelsize'] = 14

%matplotlib inline

In [2]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

In [3]:
# From book
# Pipeline component: select subsets of attributes
class DataFrameSelector(BaseEstimator, TransformerMixin):
    def __init__(self, attribs):
        self.attribs = attribs
    def fit(self, x, y=None):
        return self
    def transform(self, X):
        return X[self.attribs].values

# Pipeline component: New transformer class: drop all rows that contain invalid values
class DataSampleDropper(BaseEstimator, TransformerMixin):
    def __init__(self):
        pass
    def fit(self, x, y=None):
        return self
    def transform(self, X):
        return X.dropna(how = 'any')

# Pipeline component: Compute derivatives
class ComputeDerivative(BaseEstimator, TransformerMixin):
    def __init__(self, attribs, dt=1.0, prefix='d_'):
        self.attribs = attribs
        self.dt = dt
        self.prefix = prefix
    def fit(self, x, y=None):
        return self
    def transform(self, X):
        # Compute derivatives
        Xout = X.copy()
        for field in self.attribs:
            # Extract the values for this field
            values = Xout[field].values
            # Compute the difference between subsequent values
            diff = values[1:] - values[0:-1]
            # Bring the length to be the same as original data
            np.append(diff, 0)
            # Name of the new field
            name = self.prefix + field
            # 20 ms time step
            Xout[name] = pd.Series(diff / self.dt)
        return Xout

In [5]:
## Support for identifying kinematic variable columns
def get_kinematic_properties(data):
    # Regular expression for finding kinematic fields
    px = re.compile("_[xyz]$")

    # Find the list of kinematic fields
    fields = list(data)
    fieldsKin = [x for x in fields if px.search(x)]
    return fieldsKin

def position_fields_to_position_and_velocity_fields(fields, prefix='d_'):
    '''
    Given a list of position columns, produce a new list
    of columns that include both position and velocity
    '''
    fields_new = [prefix + x for x in fields]
    return fields + fields_new


## Load and organize data

In [10]:
# Note: you may need to change this path to get to the data
#fname = '/home/fagg/datasets/baby1/subject_k2_w10.csv'
fname = '/content/drive/MyDrive/MLP_2021/datasets/baby1/subject_k2_w10.csv'
infant_data = pd.read_csv(fname)

In [11]:
time = infant_data['time'].values
action = infant_data['sippc_action'].values

In [12]:
fieldsKin = get_kinematic_properties(infant_data)
fieldsKinVel = position_fields_to_position_and_velocity_fields(fieldsKin)


## Create Pipelines

In [13]:
prepipe = Pipeline([
    ('derivative', ComputeDerivative(fieldsKin, dt=.02)),
    ('dropper', DataSampleDropper())
])

# Position pipe
pipe_pos = Pipeline([('selector', DataFrameSelector(fieldsKin))])

# Position + velocity selector
pipe_pos_vel = Pipeline([('selector', DataFrameSelector(fieldsKinVel))])

# Robot action
attribs_label = ['sippc_action']
pipe_label = Pipeline([('selector', DataFrameSelector(attribs_label))])

# Time
attribs_time = ['time']
pipe_time = Pipeline([('selector', DataFrameSelector(attribs_time))])


In [14]:
infant_data2 = prepipe.fit_transform(infant_data)

# Selection
inputs_pos = pipe_pos.transform(infant_data2)
inputs_pos_vel = pipe_pos_vel.transform(infant_data2)
action = pipe_label.transform(infant_data2).reshape((-1,))
time = pipe_time.transform(infant_data2)

## Create class labels


In [15]:
label_motion = action > 0

# Action onset (any action)

label_assistance_onset_any = (action[0:-1] == 0) & (action[1:] > 0) & (action[1:] <= 8)
label_assistance_onset_any = np.append(label_assistance_onset_any, 0)

# Action onset: power steering
label_assistance_onset_ps = (action[0:-1] == 0) & (action[1:] > 0) & (action[1:] <= 4)
label_assistance_onset_ps = np.append(label_assistance_onset_ps, 0)

# Action onset: gesture
label_assistance_onset_g = (action[0:-1] == 0) & (action[1:] >= 5) & (action[1:] <= 8)
label_assistance_onset_g = np.append(label_assistance_onset_g, 0)


## Basic classifer

In [16]:
# All other classes
c0 = [0, 1, 2, 3, 4, 5, 6]
#  gesture
c1 = [5, 6, 7, 8]      
# None
c2 = []

label_type = np.array([0 if a in c0 else 1 if a in c1 else 2 for a in action])

ins = inputs_pos_vel
#outs = label_type
outs = label_assistance_onset_g


## Multi-class

In [58]:
ins = inputs_pos_vel
#outs = label_assistance_onset_g
outs = action 